In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys
import pandas as pd
from pandasql import sqldf
import re

path = r"C:\Users\ShubhamGupta\Downloads\diabetes-dev.csv"

df = pd.read_csv(path)  

# Get the column names
column_names = df.columns

# Get the first row values
first_row_values = df.iloc[0]

# Calculate the total number of characters in the column names
total_chars_in_column_names = sum(len(name) for name in column_names)

# Calculate the total number of characters in the first row
total_chars_in_first_row = sum(len(str(value)) for value in first_row_values)

chunk_size = int((total_chars_in_column_names + total_chars_in_first_row)*1.4)


DB_FAISS_PATH = r"C:\Users\ShubhamGupta\db_faiss"
loader = CSVLoader(file_path=path, encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
# print(data)

# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
text_chunks = text_splitter.split_documents(data)

print(len(text_chunks))

for chunk in text_chunks:
    print(chunk,"\n")


5000
page_content='PatientID: 1142956\nPregnancies: 1\nPlasmaGlucose: 78\nDiastolicBloodPressure: 41\nTricepsThickness: 33\nSerumInsulin: 311\nBMI: 50.79639151\nDiabetesPedigree: 0.420803683\nAge: 24\nDiabetic: 0' metadata={'source': 'C:\\Users\\ShubhamGupta\\Downloads\\diabetes-dev.csv', 'row': 0} 

page_content='PatientID: 1823377\nPregnancies: 0\nPlasmaGlucose: 116\nDiastolicBloodPressure: 92\nTricepsThickness: 16\nSerumInsulin: 184\nBMI: 18.60362975\nDiabetesPedigree: 0.131156495\nAge: 22\nDiabetic: 0' metadata={'source': 'C:\\Users\\ShubhamGupta\\Downloads\\diabetes-dev.csv', 'row': 1} 

page_content='PatientID: 1916381\nPregnancies: 8\nPlasmaGlucose: 171\nDiastolicBloodPressure: 42\nTricepsThickness: 29\nSerumInsulin: 160\nBMI: 35.48224692\nDiabetesPedigree: 0.082671083\nAge: 22\nDiabetic: 1' metadata={'source': 'C:\\Users\\ShubhamGupta\\Downloads\\diabetes-dev.csv', 'row': 2} 

page_content='PatientID: 1247480\nPregnancies: 3\nPlasmaGlucose: 108\nDiastolicBloodPressure: 63\nTric

In [2]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [3]:
# Converting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

In [45]:
docsearch.save_local(DB_FAISS_PATH)

In [4]:
patients = df

In [5]:
#query = "What is the value of GDP per capita of Finland provided in the data?"

#docs = docsearch.similarity_search(query, k=3)

#print("Result", docs)

llm = CTransformers(model=r"C:\Users\ShubhamGupta\Downloads\llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type="llama",
                    config={"max_new_tokens": 2048, "context_length": 4096 , "temperature": 0})

qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever(), verbose= True)


In [ ]:

chat_history = []
#     query = "What is this dataset about?"
query = input(f"Input Prompt: ")
if query == 'exit':
    print('Exiting')
    sys.exit()

edited_query = "Write a SQL query to find " + query 


result = qa({"question": edited_query, "chat_history":chat_history})
Response = result['answer']

print("Response: ",Response)
    


Input Prompt: the number of duplicate records in this dataset. A record is duplicate if the row/record appears multiple times in the dataset,ie, all the values in the row is duplicated in some other row


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

PatientID: 1204763
Pregnancies: 2
PlasmaGlucose: 104
DiastolicBloodPressure: 93
TricepsThickness: 26
SerumInsulin: 156
BMI: 39.05695763
DiabetesPedigree: 0.185593645
Age: 21
Diabetic: 1

PatientID: 1128763
Pregnancies: 1
PlasmaGlucose: 106
DiastolicBloodPressure: 91
TricepsThickness: 12
SerumInsulin: 332
BMI: 33.19366952
DiabetesPedigree: 0.187818027
Age: 31
Diabetic: 0

PatientID: 1534993
Pregnancies: 3
PlasmaGlucose: 148
DiastolicBloodPressure: 82
TricepsThickness: 44
SerumInsulin: 168
BMI: 38.10960098
DiabetesPedigree:

In [11]:
# Regular expression pattern to match SQL queries enclosed in quotes
pattern = r'(?i)(SELECT.*?;)'

# Find all matches of the pattern in the input string
matches = re.findall(pattern, Response, re.IGNORECASE)

# Print all SQL queries found
for match in matches:
    print(match)




SELECT * FROM patients WHERE (SELECT COUNT(*) FROM patients GROUP BY PatientID) = 2;


In [12]:
# Use pandasql to execute the SQL query
result = sqldf(matches[0], locals())
# Print the result
if( result.empty ):
    print("No records")
    
else:
    print(result)

No records


In [56]:
# while(True):
    
chat_history = []
#     query = "What is this dataset about?"
query = input(f"Input Prompt: ")
if query == 'exit':
    print('Exiting')
    sys.exit()

edited_query = "Write a SQL query to find " + query + ". The table contains many records like mentioned above."


result = qa({"question": edited_query, "chat_history":chat_history})
Response = result['answer']

print("Response: ",Response)


# Regular expression pattern to match SQL queries enclosed in quotes
pattern = r'(?i)(SELECT.*?;)'

# Find all matches of the pattern in the input string
matches = re.findall(pattern, Response)



Input Prompt: Find duplicate records


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

PatientID: 1204763
Pregnancies: 2
PlasmaGlucose: 104
DiastolicBloodPressure: 93
TricepsThickness: 26
SerumInsulin: 156
BMI: 39.05695763
DiabetesPedigree: 0.185593645
Age: 21
Diabetic: 1

PatientID: 1128763
Pregnancies: 1
PlasmaGlucose: 106
DiastolicBloodPressure: 91
TricepsThickness: 12
SerumInsulin: 332
BMI: 33.19366952
DiabetesPedigree: 0.187818027
Age: 31
Diabetic: 0

PatientID: 1562563
Pregnancies: 9
PlasmaGlucose: 151
DiastolicBloodPressure: 63
TricepsThickness: 39
SerumInsulin: 366
BMI: 31.29822172
DiabetesPedigree: 1.552917234
Age: 42
Diabetic: 1

PatientID: 1822695
Pregnancies: 3
PlasmaGlucose: 106
DiastolicBloodPressure: 60
TricepsThickness: 45
SerumInsulin: 355
BMI: 33.54852

In [55]:
matches = re.findall(pattern, Response)

# Print all SQL queries found
for match in matches:
    print(match)

In [52]:
# Use pandasql to execute the SQL query
result = sqldf(matches[0], locals())

# Print the result
if( result.empty ):
    print("No records")

result.head(len(result))


PandaSQLException: (sqlite3.OperationalError) misuse of aggregate function COUNT()
[SQL: SELECT * FROM patients WHERE COUNT(*) > 1;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [51]:
patients = df